In [56]:
import pandas as pd
import numpy as np

In [16]:
file_path_orders_payment = r'C:\Users\mathe\Projects\clients_seg_database\olist_order_payments_dataset.csv'
file_path_orders = r'C:\Users\mathe\Projects\clients_seg_database\olist_orders_dataset.csv'
file_path_customers = r'C:\Users\mathe\Projects\clients_seg_database\olist_customers_dataset.csv'

In [18]:
df_orders_payment = pd.read_csv(file_path_orders_payment)
df_orders = pd.read_csv(file_path_orders)
df_customers = pd.read_csv(file_path_customers)

In [20]:
df_orders_payment_selected = df_orders_payment[['order_id', 'payment_type', 'payment_installments', 'payment_value']]
df_orders_selected = df_orders[['order_id', 'customer_id', 'order_approved_at']]
df_customers_selected = df_customers[['customer_id', 'customer_unique_id']]

In [27]:
df_merge = pd.merge(
    df_orders_payment_selected,
    df_orders_selected,
    left_on='order_id',
    right_on='order_id',
    how='inner'
)

In [64]:
df_merge_2 = pd.merge(
    df_merge,
    df_customers_selected,
    left_on='customer_id',
    right_on='customer_id',
    how='inner'
)

In [65]:
df_merge_2['order_approved_at'] = pd.to_datetime(df_merge_2['order_approved_at'])
df_merge_2 = df_merge_2[df_merge_2['payment_type'] == 'credit_card']
df_merge_2 = df_merge_2[df_merge_2['order_approved_at'].notna()]

In [66]:
df_recencia_index = df_merge_2.groupby('customer_unique_id', as_index=False)['order_approved_at'].max()
df_recencia_index = df_recencia_index.sort_values('order_approved_at', ascending=False)
df_recencia_index['index'] = range(1, len(df_recencia_index) + 1)

In [68]:
condicoes = [
    (df_recencia_index['index'] <= (len(df_recencia_index) * 0.2)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.2)) & (df_recencia_index['index'] <= (len(df_recencia_index) * 0.4)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.4)) & (df_recencia_index['index'] <= (len(df_recencia_index) * 0.6)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.6)) & (df_recencia_index['index'] <= (len(df_recencia_index) * 0.8)),
    (df_recencia_index['index'] > (len(df_recencia_index) * 0.8))
]

valores = [
    '5',  
    '4',  
    '3',
    '2',
    '1'    
]

df_recencia_index['recencia_score'] = np.select(condicoes, valores, default='Outros')

In [69]:
df_frequency = pd.DataFrame(
    df_merge_2['customer_unique_id'].value_counts()
).reset_index()

df_frequency.columns = ['customer_unique_id', 'orders_count']

In [97]:
df_order_quantity_list = (
    df_frequency['orders_count']
    .value_counts()               
    .reset_index()                
    .rename(columns={
        'count': 'customers_count'
    })
    .sort_values('orders_count', ascending=False)  
)


In [99]:
condicoes = [
    (df_order_quantity_list['orders_count'] <= (len(df_order_quantity_list) * 0.2)),
    (df_order_quantity_list['orders_count'] > (len(df_order_quantity_list) * 0.2)) & (df_order_quantity_list['orders_count'] <= (len(df_order_quantity_list) * 0.4)),
    (df_order_quantity_list['orders_count'] > (len(df_order_quantity_list) * 0.4)) & (df_order_quantity_list['orders_count'] <= (len(df_order_quantity_list) * 0.6)),
    (df_order_quantity_list['orders_count'] > (len(df_order_quantity_list) * 0.6)) & (df_order_quantity_list['orders_count'] <= (len(df_order_quantity_list) * 0.8)),
    (df_order_quantity_list['orders_count'] > (len(df_order_quantity_list) * 0.8))
]

valores = [
    '1',  
    '2',  
    '3',
    '4',
    '5'    
]

df_order_quantity_list['frequency_score'] = np.select(condicoes, valores, default='Outros')

In [102]:
df_frenquency_index = df_frequency.merge(
    df_order_quantity_list,
    on='orders_count',
    how='left'
)
df_frenquency_index = df_frenquency_index[['customer_unique_id', 'orders_count', 'frequency_score']]

In [105]:
print(df_frenquency_index)

                     customer_unique_id  orders_count frequency_score
0      8d50f5eadf50201ccdcedfb9e2ac8455            17               5
1      3e43e6105506432c953e165fb2acf44c             9               5
2      6469f99c1f9dfae7733b25662e7f1782             7               4
3      dc813062e0fc23409cd255f7f53c7074             6               4
4      63cfc61cee11cbe306bff5857d00bfe4             6               4
...                                 ...           ...             ...
74050  107df8fcfe13d1a58006f0a11dc7522c             1               1
74051  6608c37202accfe3cda3fca4c8a16062             1               1
74052  3146229460ce28f41121b47929b046b0             1               1
74053  8247de12f35a00cf569c5b6a00a746ec             1               1
74054  9a3526eee723accd014ec683afe5bf20             1               1

[74055 rows x 3 columns]
